# Include the libraries

In [13]:
from __future__ import division
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', 800)
import MySQLdb
import urllib2
import datetime
from ipywidgets import interact, interactive, fixed
from bokeh.charts import Bar, output_notebook, show
import json
import re


from read_sql_table import *
from read_survey_question import *
from read_survey_response import *
from read_json_response import *
from read_json_weights import *
from computing_each_question import *
from plot_geo_address import *
from plot_time_serial import *
from computing_survey_metrics import *
from computing_open_text import *
from clear_format_tags import *

# Set the survey_id and running time

In [14]:
from ipywidgets import *
import ipywidgets
import calendar
from IPython.display import display

In [15]:
# repeatable = widgets.RadioButtons(
#     options=['repeatable', 'non-repeatable'],
#     description='condition',
#     disabled=False
# )

# survey_id = IntSlider(value = 242, min = 1, max = 300, step = 1, description = 'survey id')
# b1 = Button(description='Enter the Survey Id')
# b1.button_style = 'success'
# display(b1)
# display(VBox([survey_id]))
# display(repeatable)

# SurveyId = survey_id.value
# if repeatable.value == 'repeatable':
#     repeatable = True
# else:
#     repeatable = False

# start_year = IntSlider(value = 2017, min = 2016, max = 2018, step = 1, description = 'Year')
# start_month = IntSlider(value = 4, min = 1, max = 12, step = 1, description = 'Month')
# start_day = IntSlider(value = 1, min = 1, max = 31, step = 1, description = 'Day')

# end_year = IntSlider(value = 2017, min = 2016, max = 2018, step = 1, description = 'Year')
# end_month = IntSlider(value = 5, min = 1, max = 12, step = 1, description = 'Month')
# end_day = IntSlider(value = 1, min = 1, max = 31, step = 1, description = 'Day')

# b1 = Button(description='Enter the start date')
# b1.button_style = 'info'
# display(b1)
# display(HBox([start_year]))
# display(HBox([start_month, start_day]))

# b1 = Button(description='Enter the end date')
# b1.button_style = 'primary'
# display(b1)
# display(HBox([end_year]))
# display(HBox([end_month, end_day]))

# start_date = datetime.datetime(start_year.value, start_month.value, start_day.value)
# end_date = datetime.datetime(end_year.value, end_month.value, end_day.value)

# start_time = pd.tslib.Timestamp(start_date)
# end_time = pd.tslib.Timestamp(end_date)
# if end_time < start_time:
#     raise ValueError('The start time is later than the end time')
# elif start_time > pd.tslib.Timestamp(datetime.datetime.utcnow()):
#     raise ValueError('The start time is later than the current time')

In [16]:
SurveyId = 242
start_time = pd.tslib.Timestamp('2017-04-01 00:00:00')
# end_time = pd.tslib.Timestamp('2017-03-01 00:00:00')
end_time = pd.tslib.Timestamp(datetime.datetime.utcnow())
repeatable = True

# Get all the survey responses

In [17]:
survey_questions = get_survey_question(SurveyId)

survey_response_unique = get_survey_response(SurveyId, start_time, end_time, repeatable)
result = get_json_response(survey_questions, survey_response_unique['jsonResponse'].tolist())
result = pd.concat([result, survey_response_unique.drop(['jsonResponse'], axis = 1)], axis = 1)

# calculate the weights and total_score for a screener
weights = get_json_weight(survey_questions, survey_response_unique['jsonResponse'].tolist())
result['total_score_for_screener'] = weights.sum(axis = 1)

result, survey_questions = clear_html_tag(result, survey_questions)

In [18]:
w = widgets.ToggleButtons(
    options=['result', 'questions'],
    description='data',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)

w1 = widgets.ToggleButtons(
    options=['metrics', 'data', 'data_download'],
    description='Options',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
#     icon='check'
)
@interact(w = w, w1 = w1, __manual=True)
def click_me(w, w1):
    if w == 'result':
        if w1 == 'metrics':
            display(get_survey_metrics(SurveyId, start_time, end_time, result))
        elif w1 == 'data':
            display(result)
        elif w1 == 'data_download':
            result.to_csv('result_%d_%s.csv' % (SurveyId, end_time))
            print 'File is created: result_%d_%s.csv' % (SurveyId, end_time)
            
            
    elif w == 'questions':
        if w1 == 'metrics':
            print 'There are %d questions in the survey' % len(survey_questions)
        elif w1 == 'data':
            display(survey_questions)
        elif w1 == 'data_download':
            survey_questions.to_csv('survey_questions_%d.csv' % SurveyId)
            print 'File is created: survey_questions_%d.csv' % SurveyId
    
    

File is created: survey_questions_242.csv


# Summary for each question

In [19]:
w2 = widgets.Dropdown(
    options=[str(item) for item in survey_questions['id'].values ],
#     value='2',
    description='Q id:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
@interact(w2 = w2, __manual=True)
def Q_summary(w2):
    temp = summary_for_each_question(survey_questions, result, w2)
    display(temp)

In [20]:
w3 = widgets.Dropdown(
    options=[item for item in survey_questions['id'].values ],
#     value='2',
    description='Q id:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)
@interact(w3 = w3, __manual=True)
def Q_plot(w3):
    temp = summary_plot_for_each_question(survey_questions, result, w3)
    display(temp)


In [21]:
# interact(summary_for_each_question, survey_questions = fixed(survey_questions), result = fixed(result), \
#         question_id = [item for item in survey_questions['id'].values ])

In [22]:
# interact(summary_plot_for_each_question, survey_questions = fixed(survey_questions), result = fixed(result), \
#         question_id = [item for item in survey_questions['id'].values ], __manual = True)

# Survey geo metrics

In [23]:
w4 = widgets.ToggleButton(
    value=False,
    description='make Geo plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)
@interact(w4 = w4)
def geo_plot(w4):
    if w4 == True:
        display(plot_geo(result['latitude'], result['longitude']))
    else:
        print 'Silent mode'

Silent mode


# Survey time serial

In [24]:
result = UTC_to_EST(result)
# survey_response_analyze_filter_hour(result, 'day')
traffic = interact(survey_response_analyze_filter, result = fixed(result), time_difference = ['hour', 'day'])


Loading BokehJS ...

,time_stamp,daily_count
0,2017-04-05 14:00:00,63
1,2017-04-06 16:00:00,58
2,2017-04-05 17:00:00,57
3,2017-04-05 16:00:00,56
4,2017-04-06 15:00:00,54
5,2017-04-05 15:00:00,51
6,2017-04-10 15:00:00,50
7,2017-04-05 13:00:00,50
8,2017-04-05 19:00:00,49
9,2017-04-05 20:00:00,47
